# Actividad 3. Algoritmo _Apriori_
<hr/>

**Profesor:** 
- Mauricio Arriagada

**Ayudantes:**
- Felipe Barrientos (fnbarrientos@uc.cl)
- Javier Dreves (jidreves@uc.cl)
- Joaquin Eichholz (jeichholz3@uc.cl)
- Astrid San Martín (aesanmar@uc.cl)


**Antes de comenzar:**

- Laboratorio debe ser realizado **de forma individual**. Obviamente, se pueden discutir ideas, pero cualquier intercambio de códigos **no está permitido**.
- Recuerda orientar tu programación a un paradigma funcional.
- **¡ Comenta todo tu código !**

**Instrucciones de entrega:**

- Debe entregar este laboratorio por `siding`, en sección `cuestionarios`. Descargar archivo ".ipynb" a su equipo y luego subirlo. (También pueden trabajarlo en colab)
- Plazo máximo de entrega: **Martes 3 de Agosto, 10.59 pm.**

**Evaluación:**
- La estructura del código = 0.5
- Salida exitosa = 0.5


<hr/>

**Contexto**

Este laboratorio consiste en estudiar en profundidad e implementar el algoritmo `Apriori`, el cual tiene por
objetivo encontrar `itemsets` frecuentes dentro de una base de datos y generar las reglas de asociación que
superan umbrales de soporte y confianza.
En `data mining`, las reglas de asociación son ampliamente utilizadas para descubrir relaciones entre
variables en bases de datos de gran tamaño [Agrawal et al., 1993]. Aplicaciones clásicas de este tipo de
estrategias pueden ser encontradas en análisis de compras y de características socio-demográficas desde
bases de datos censales.

**Base de Datos**

La base de datos a utilizar corresponde a una parte de la información liberada por `Spotify` para el _RecSys_ Challenge 2018 1 y contiene información de listas de reproducción creadas por usuarios de `Spotify`. Cada lista contiene un número variable de canciones 

**Objetivo**

Para este laboratorio, aplicaremos el algoritmo `a priori` en una base de datos de listas de reproducción publicada por `Spotify`. El objetivo es obtener itemsets de canciones frecuentes y crear reglas de asociación con estos sets. 


<hr/>

## Instrucciones 

1. **Implementar el algoritmo Apriori**: Se espera que el algoritmo sea capaz de retornar a lo menos 3 reglas de asociación entre canciones. El alumno debe ir probando con distintos humbrales  para el la ejecución del código tome un tiempo razonable pero se puedan tener reglas no triviales. 
    - Cargar base de datos
    - Primera pasada: Eliminar todas las listas que contienen canciones que no pasan el umbral
    - Iteraciones posteriores: Generar itemsets con canciones restantes y ver su presencia en las listas que van quedando 


2. **Presentación de reglas**: Mostrar las reglas encontradas y explicar por qué se eligiron. Se deben incluir conceptos de _support_ y _confidence_ de cada regla. 

## Recordar 

- Support = |Itemset| / |T|
    - S = |(pañales, leche, cerveza)|

- Confidence = |Itemset| / |Itemset - regla|
    - C = |(pañales, leche, cerveza)| / |(leche, pañales)|


- **Es de suma importancia que el alumno elabore todo el algoritmo desde cero. No está permitido reutilizar código o usar librerías para la implementación.**

In [4]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [5]:
def load_database(path):
    
    """
    Esta función debe cargar la base de datos de acuerdo a un path. Esta función puede servir también 
    como filtro del número de listas de reproducción a usar para probar a priori.
    """
    data = np.load(path)
    database = np.array([x for x in data.item().values()])
    return database

In [6]:
def ordenar(data):
    """
    Esta función define el orden que será utilizado cuando se hagan los joins 
    """
    canciones = {}
    contador = 0
    for lista in data:
        for cancion in lista:
            if cancion not in canciones:
                canciones[cancion]= contador
                contador += 1
    return canciones

In [7]:
def calcular_soporte(data, itemset=None):
    """
    Esta función saca el soporte de los itemsets
    """
    t = []
    canciones = defaultdict(int)
    
    if itemset == None: 
        # si es la primera iteración, cuenta las diferentes canciones y sus apariciones
        for lista in data:
            for cancion in lista:
                t.append(cancion)
                canciones[tuple(t)] += (1/len(data))
                t = []
    else:
        # si es una iteración diferente cuenta que este el set completo.
        for lista in data:
            for item in itemset:
                if all(x in lista for x in item): # si todos los items estan en la lista
                    canciones[item] += (1/len(data))
                    
    return canciones

In [8]:
def generar_join(items, orden):
    """
    Esta función genera los joins entre los sets deacuerdo al orden dado.
    """
    itemset= []
    
    for item in items.keys():
        for seg in items.keys():
            if item != seg:
                
                if len(item)==1: #si solo necesitamos formar pares
                    """ revisamos cual de los 2 es mayor e insertamos en orden"""
                    if orden[item[0]]< orden[seg[0]]:  
                        tupla = (item[0],seg[0])
                    else:
                        tupla = (seg[0], item[0])
                    itemset.append(tupla)
                    
                else: # si vamos a generar tuplas de más de largo 2.
                    cont = 0
               
                    for el1, el2 in zip(item, seg):
                        cont +=1
                        if (el1 != el2 and cont == len(item) ):
                            """
                            si hay un elemento diferente, es el ultimo elemento hacemos join considerando 
                            el orden inicial dado
                            """
                            
                            if ((orden[item[cont-1]] < orden[seg[cont-1]])): 
                                tupla = item + tuple([seg[cont-1]])
                            else:
                                tupla = seg + tuple([item[cont-1]])
                            itemset.append(tupla)
                                
                        elif (el1 != el2 and cont != len(item) ):
                            break
                        else:
                            pass
                                
    return itemset
                        
            

In [9]:
def apriori(data, support):
    
    """
    Función que debe retornar todos los itemsets que cumplen con el support. Puede ser en una estructura de 
    diccionario que entregue el itemset y su soporte.
    """
    itemsets = {}
    #primero calculo el soporte para los items por si solos 
    soporte = calcular_soporte(data)
    """Ya que no se puede deducir nada de un item por sí solo no lo agregué a la lista de items, está comentado 
    aquí abajo. """
    # itemsets.update({key: soporte[key] for key in soporte.keys() if soporte[key]>= support})
    items = generar_join({key: soporte[key] for key in soporte.keys() if soporte[key]>= support}, orden)
    
    #Luego realizo un while, mientras se puedan generar mas posibles itemsets 
    while(items!= []):
        soporte= calcular_soporte(data, items)
        itemsets.update({key: soporte[key] for key in soporte.keys() if soporte[key]>= support})
        items = generar_join({key: soporte[key] for key in soporte.keys() if soporte[key]>= support}, orden)
    

    
    return itemsets

In [38]:
def confianza(data, antecedente, consecuente):
    """Función que calcula la confianza de acuerdo a las diapositivas."""
    arriba = tuple(antecedente) + tuple(consecuente)
    freq_up = 0 
    freq_down = 0
    for lista in data:
        if all(x in lista for x in arriba):
            freq_up += 1
        if all(j in lista for j in tuple(antecedente)):
            freq_down += 1
    if freq_down == 0:
        return 0
    return (freq_up/freq_down)

In [11]:
def obtener_subsets(item):
    subs = [[]]
    """Función que obtiene todos los subsets de un item"""
    for elemento in item:
        for sub_set in subs:
            subs = subs + [list(sub_set)+ [elemento]]
    return [set(i) for i in subs if len(set(i))!=0 ]

In [23]:
def rules(itemsets, confidence, data):
    
    """
    Retorna las reglas que complen con la confianza pedida.
    """
    rules= [] # rules es un diccionario que como key tiene el antecedente 
            # y value el consecuente.
    for item in itemsets.keys():
        set1 = set(item)
        subsets = obtener_subsets(item)
        for subset in subsets:
             if len(set1-subset) != 0:
                    conf = confianza(data, set1-subset, subset)
                    if conf >= confidence:
                        rules.append({'antecedente': tuple(set1-subset), 'consecuente': tuple(subset),'confianza': conf, 'soporte': itemsets[item] })
    return rules 

In [13]:
data = load_database("spotify.npy")
datita = data[0:5000]
orden = ordenar(datita) # defino un orden para mi dataset

itemsets = apriori(datita, 0.025) 
#defini este soporte minimo ya que con los 10000 datos era uno de los que 
#arrojaba datos suficientes para ver el algoritmo funcionar, no sé que valor habría sido más conveniente.


In [14]:
itemsets

{('Despacito - Remix', "I'm the One"): 0.027199999999999946,
 ('Bounce Back', 'goosebumps'): 0.02679999999999995,
 ('Bounce Back', 'Congratulations'): 0.029199999999999934,
 ('Bounce Back', 'XO TOUR Llif3'): 0.02519999999999996,
 ('goosebumps', 'Congratulations'): 0.03319999999999991,
 ('goosebumps', 'XO TOUR Llif3'): 0.031199999999999922,
 ('Congratulations', 'XO TOUR Llif3'): 0.037599999999999884,
 ('XO TOUR Llif3', 'Bank Account'): 0.02999999999999993,
 ('Caroline', 'Fake Love'): 0.02839999999999994,
 ('Bounce Back', 'Caroline'): 0.02839999999999994,
 ('Bounce Back', 'Fake Love'): 0.02639999999999995,
 ('Congratulations', 'Caroline'): 0.025599999999999956,
 ('Congratulations', 'Fake Love'): 0.025599999999999956,
 ('Closer', 'Gold'): 0.025999999999999954,
 ('goosebumps', 'HUMBLE.'): 0.031199999999999922,
 ('Congratulations', 'HUMBLE.'): 0.043999999999999845,
 ('Congratulations', 'Tunnel Vision'): 0.027599999999999944,
 ('Passionfruit', 'HUMBLE.'): 0.029199999999999934,
 ('HUMBLE.', '

# Reglas finales
Estas reglas que se presentan a continuación fueron tomadas de cada itemset que no era de un solo elemento y que pasó el humbral de dado,
finalmente, luego de tener cada itemset, se generaron todas las posibles reglas y nos quedamos con las que sobrepasan o son iguales auna confianza del 60%

In [42]:
reglas = rules(itemsets, 0.6, data)
reglas

[{'antecedente': ('XO TOUR Llif3',),
  'consecuente': ('HUMBLE.',),
  'confianza': 0.631578947368421,
  'soporte': 0.03959999999999987},
 {'antecedente': ('Mask Off',),
  'consecuente': ('HUMBLE.',),
  'confianza': 0.6455696202531646,
  'soporte': 0.040799999999999864},
 {'antecedente': ('Cold Water (feat. Justin Bieber & MØ)',),
  'consecuente': ('Closer',),
  'confianza': 0.6196581196581197,
  'soporte': 0.0347999999999999},
 {'antecedente': ('HUMBLE.', 'iSpy (feat. Lil Yachty)'),
  'consecuente': ('Congratulations',),
  'confianza': 0.7032258064516129,
  'soporte': 0.025599999999999956},
 {'antecedente': ('Congratulations', 'iSpy (feat. Lil Yachty)'),
  'consecuente': ('HUMBLE.',),
  'confianza': 0.6728395061728395,
  'soporte': 0.025599999999999956},
 {'antecedente': ('Mask Off', 'iSpy (feat. Lil Yachty)'),
  'consecuente': ('HUMBLE.',),
  'confianza': 0.6870748299319728,
  'soporte': 0.02519999999999996},
 {'antecedente': ('HUMBLE.', 'iSpy (feat. Lil Yachty)'),
  'consecuente': ('

<hr/>
<center> <h1>Fin del laboratorio.</h1> </center>
​